# ChipWhisperer

> ChipWhisperer is a complete open source toolchain for learning about side channel attacks on embedded devices and validating the side channel resistance of these devices.

-- https://chipwhisperer.readthedocs.io/en/latest/getting-started.html#what-is-chipwhisperer

![](https://wiki.newae.com/images/5/54/Cwlite_basic.png)

### ChipWhisperer-Lite XMEGA

- Low-Noise Amplifier (LNA) with adjustable gain of up to +60 dB for analog power measurements.
- 10-bit ADC up to 105 MS/s, with ultra flexible clocking mechanism allow synchronous power capture.
- Generate clock glitches with sub-nS resolution on timing.
- Built-in serial port for communications and programming STM32Fx targets.
- Built in XMEGA and AVR programmers.
- Open-source design allowing you to modify all aspects of board (FPGA, microcontroller, etc).
- ATXMEGA128D4 8-bit RISC target for power analysis & fault injection integrated onto board.
- Can separate target off board to use tool on additional targets.
- ~250$


### Components

- Hardware
  https://rtfm.newae.com/
- Firmware
  https://chipwhisperer.readthedocs.io/en/latest/
- Software
  https://chipwhisperer.readthedocs.io/en/latest/
- Tutorials
  https://github.com/newaetech/chipwhisperer-jupyter

## Example

### Initialization

In [ ]:
import securec
import securec.util
scope, target = securec.util.init()

In [ ]:
scope

### First code example

```c
#include "hal.h"
#include "simpleserial.h"
#include <stdint.h>
#include <string.h>

uint8_t hello(uint8_t cmd, uint8_t scmd, uint8_t len, uint8_t *buf)
{
    simpleserial_put(0x01, 12, "hello world.");
    return 0;
}

int main(void)
{
    platform_init();
    init_uart();
    trigger_setup();

    simpleserial_init();
    simpleserial_addcmd(0x01, 0, hello);
    while (1)
        simpleserial_get();
}
```

In [ ]:
securec.util.compile_and_flash('./example_1.c')

In [ ]:
securec.util.reset_target()
target.simpleserial_write(0x01, b'')
bytes(target.simpleserial_read(0x01))

### Capturing a trace

Add a trigger point:

```c
uint8_t hello(uint8_t cmd, uint8_t scmd, uint8_t len, uint8_t *buf)
{
    trigger_high();

    simpleserial_put(0x01, 12, "hello world.");

    trigger_low();
    return 0;
}
```

In [ ]:
scope.default_setup()
scope.adc.samples = 1000
scope.arm()
target.simpleserial_write(0x01, b'')
trace = securec.util.capture()

In [ ]:
trace

### Plot the trace

In [ ]:
from bokeh.plotting import figure, show 
from bokeh.io import output_notebook
from bokeh.models import CrosshairTool

output_notebook()

In [ ]:
p = figure(height=300, sizing_mode='stretch_width')
p.add_tools(CrosshairTool())
p.line(range(0, len(trace)), trace)
show(p)